# Unsupervised Learning (Matrix Factorization)

In [1]:
# Import libraries needed
import pandas as pd
from sklearn.model_selection import train_test_split

## Data setup

In [2]:
# Read dataset and create training an testing datasets
df = pd.read_csv('./Data/BBC News Train.csv')
feature_data = df.drop(columns=["Category"])
X_train, X_test, Y_train, Y_test = train_test_split(feature_data, df["Category"], test_size=0.2)

## Support functions

## Model Creation

## Kaggle Test Processing